# Linear spectra of a dimer system

Importing useful classes, methods and modules:

In [ ]:
from qudofedi import Qjob, FeynmanDiagram, System, PostProcess
from qutip import sigmax, sigmaz, sigmay, tensor, identity
from qiskit import Aer
import numpy as np

System parameters:

In [ ]:
e_1 = 1.55                                   # Electronic energy gap for monomer 1 [eV]
e_2 = 1.46                                   # Electronic energy gap for monomer 2 [eV]
J = -0.01                                    # Exciton coupling [eV]
H = (- e_1 / 2 * tensor(sigmaz(), identity(2))
     - e_2 / 2 * tensor(identity(2), sigmaz())
     + J / 2 * ( tensor(sigmax(), sigmax())
                + tensor(sigmay(), sigmay()))
    )                                        # Hamiltonian of the dimer system [eV]

mu_1 = 1.                                    # Electric dipole moment for monomer 1 [a.u.]
mu_2 = 1.                                    # Electric dipole moment for monomer 2 [a.u.]

system = System(H, [mu_1, mu_2])             # System object

Spectroscopy parameters:

In [ ]:
t_min = 0.                                   # Initial time [rad/eV]
t_max = 201.                                 # Final time [rad/eV]
dt = 10.                                     # Delta time [rad/eV]
t_list = np.arange(t_min, t_max, dt)         # Time list [rad/eV]
                 
FD = FeynmanDiagram('a', t_list)             # FeynmanDiagram object

IBM Quantum backend:

In [ ]:
backend = Aer.get_backend('qasm_simulator')

In [ ]:
# from qiskit_ibm_provider import IBMProvider
# 
# provider = IBMProvider(instance='ibm-q/open/main')
# backend = provider.get_backend('ibmq_manila')

Create the Qjob object and getting the response function:

In [ ]:
job = Qjob(system, FD)                       # Qjob object

response = job.run(backend,
                   shots = 4000,
                   save_Qjob = True,
                   save_name = 'example_a_dimer',
                   save_checkpoint = True,
                  )                          # Run the simulation returning a numpy.ndarray with the results

Post-processing the results (rotating frame, exponential damping, zero-padding) to obtain the time oscillation and frequency spectra:

In [ ]:
PostProcess(job,
            RF_freq = 1.505,
            damping_rate = 29.54 * 10**(-3),
            pad = 3,
           )                                 # Post-process and print the results

## Adding readout-error mitigation

In [ ]:
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit_ibm_provider import IBMProvider

Select a real quantum processor:

In [ ]:
provider = IBMProvider(instance='ibm-q/open/main')
real_backend = provider.get_backend('ibmq_manila')

Create the noise model (not necessary if using the real_backend as the backend of computation):

In [ ]:
from qiskit_aer.noise.noise_model import NoiseModel

noise_model = NoiseModel().from_backend(real_backend)

Create the Qjob object and getting the response function:

In [ ]:
job = Qjob(system, FD)                       # Qjob object

response = job.run(backend,
                   shots = 4000,
                   save_Qjob = True,
                   save_name = 'example_a_dimer_mitigation',
                   save_checkpoint = True,
                   noise_model = noise_model,
                   coupling_map = real_backend.configuration().coupling_map,
                   initial_layout = [0, 1, 2],    # An initial_layout specification is necessary to not get errors in Qiskit
                   measurement_error_mitigation_cls=CompleteMeasFitter,
                   measurement_error_mitigation_shots = 4000,
                  )                          # Run the simulation returning a numpy.ndarray with the results

In [ ]:
# For readout-error mitigation on a real device:
#
# job = Qjob(system, FD)                       # Qjob object
#
# response = job.run(backend,
#                    shots = 4000,
#                    save_Qjob = True,
#                    save_name = 'example_a_dimer_mitigation',
#                    save_checkpoint = True,
#                    initial_layout = [0, 1, 2],    # An initial_layout specification is necessary to not get errors in Qiskit
#                    measurement_error_mitigation_cls=CompleteMeasFitter,
#                    measurement_error_mitigation_shots = 4000,
#                   )                          # Run the simulation returning a numpy.ndarray with the results

Post-processing the results (rotating frame, exponential damping, zero-padding) to obtain the time oscillation and frequency spectra:

In [ ]:
PostProcess(job,
            RF_freq = 1.505,
            damping_rate = 29.54 * 10**(-3),
            pad = 3,
           )                                 # Post-process and print the results